In [5]:
import tensorflow as tf
from keras import layers
from keras import Sequential
import numpy as np
import codecs
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import time
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [6]:
WORD_LEN = 25
MAX_LEN_CODE = 123.0
END_LEN = 20
F_NAME = "data_sets/dict.cc_nouns_with_gender.txt"
FILE_TEST_NAME = "data_sets/deutsch_test_asc_1.txt"
DROP_OUT = 0.2

In [7]:
def readFile(file_name, MAX_LEN_CODE, END_LEN):
    training_data_file = open(file_name, 'r')
    training_data_list = training_data_file.readlines()
    training_data_file.close()

    training_data_list = training_data_list[1:]
    print(training_data_list[:10])
    np.random.shuffle(training_data_list)
    print(training_data_list[:10])

    f_inputs = [[[0] for i in range(0, END_LEN)]]
    targets = []
    f_inputs = np.asfarray(f_inputs)

    for record in training_data_list:
        all_values = record.split(' ')
        all_values = [word.replace("\n", "") for word in all_values]
        i = [0]
        for letter in all_values[1]:
            i = np.vstack((i, np.asfarray([ord(letter.lower())/MAX_LEN_CODE])))
            pass
        inputs = i
        if(len(inputs) > END_LEN):
            inputs = inputs[len(inputs)-END_LEN:]
        elif(len(inputs) < END_LEN):
            begin = np.zeros((END_LEN - len(inputs)), dtype=np.float)
            for b in begin:
                inputs = np.vstack((b, inputs))
                pass
        f_inputs = np.vstack((f_inputs, [inputs]))

        targets.append(all_values[0])

    f_inputs = np.delete(f_inputs, 0, 0)
    encoder = LabelEncoder()
    encoder.fit(targets)
    new_targets = encoder.transform(targets)
    f_targets = np_utils.to_categorical(new_targets)
    print("Done!")

    return f_inputs, f_targets

In [ ]:
X, Y = readFile(F_NAME, MAX_LEN_CODE, END_LEN)

['die AAA-Einstufung {f}\n', 'die Aach {f}\n', 'das Aach {n}\n', 'das Aachen {n}\n', 'der Aachener {m}\n', 'die Aachenerin {f}\n', 'die Aachenfahrt {f}\n', 'die AA-Formula {f}\n', 'der Aagaard-Gletscher {m}\n', 'das Aagenaes-Syndrom {n}\n']
['die Nassalarmventilstation {f}\n', 'die Vers_hnung {f}\n', 'der Anzac {m}\n', 'die Eigent]mergemeinschaft {f}\n', 'der Fertigungsbereich {m}\n', 'die Abszissenachse {f}\n', 'die Kongestion {f}\n', 'die Sondertilgung {f}\n', 'der Fahlstirnsericornis {m}\n', 'der Fahrradreifengummi {m}\n']


In [10]:
NAME = "German_noun-2Lstm1(60,128)-ep100-{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))


model = Sequential()

model.add(layers.LSTM(60, input_shape=(X.shape[1:]), activation='relu', return_sequences=True))
model.add(layers.Dropout(DROP_OUT))
model.add(layers.LSTM(128, activation='relu'))
model.add(layers.Dense(60, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=['accuracy'])

model.fit(X[1000:], Y[1000:], epochs=100, validation_split=0.25, callbacks=[tensorboard])

prediction = model.predict_classes(X[:999])
np.set_printoptions(threshold=np.nan)
print(prediction)

scores = model.evaluate(X[:999], Y[:999], verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
print(scores)

print("\n")

Train on 280371 samples, validate on 93457 samples
Epoch 1/100
280371/280371 [==============================] - 391s 1ms/step - loss: 1.0451 - acc: 0.4563 - val_loss: 1.0022 - val_acc: 0.5298
Epoch 2/100
280371/280371 [==============================] - 382s 1ms/step - loss: 0.8992 - acc: 0.5815 - val_loss: 0.8348 - val_acc: 0.6265
Epoch 3/100
280371/280371 [==============================] - 298s 1ms/step - loss: 0.8413 - acc: 0.6174 - val_loss: 0.8082 - val_acc: 0.6358
Epoch 4/100
280371/280371 [==============================] - 298s 1ms/step - loss: 0.8002 - acc: 0.6402 - val_loss: 0.7758 - val_acc: 0.6620
Epoch 5/100
280371/280371 [==============================] - 299s 1ms/step - loss: 0.7622 - acc: 0.6641 - val_loss: 0.7268 - val_acc: 0.6889
Epoch 6/100
280371/280371 [==============================] - 299s 1ms/step - loss: 0.7304 - acc: 0.6816 - val_loss: 0.6876 - val_acc: 0.7079
Epoch 7/100
280371/280371 [==============================] - 366s 1ms/step - loss: 0.7031 - acc: 0.6970

In [11]:
answer = [list(i).index(1) for i in Y[:1000]]
score = 0
for i in range(len(answer)):
    if answer[i] == prediction[i]:
        score += 1
        pass
    pass
print(score)
print(len(answer))
result = 100 * score/len(answer)
print("{}%".format(round(result)))

[2 1 2 0 2 1 2 0 2 2 2 2 2 1 2 1 1 2 0 1 0 2 2 1 1 1 2 1 2 1 2 1 0 2 0 2 2
 1 0 2 0 1 1 1 2 1 0 2 0 2 1 1 1 1 1 2 1 2 1 1 2 1 2 2 2 1 2 1 2 0 2 1 2 0
 2 2 2 1 1 1 0 1 2 0 1 1 2 1 0 2 2 1 2 2 2 1 0 2 2 0 2 2 2 2 2 1 2 2 2 0 0
 2 2 0 1 1 1 1 1 0 1 0 2 2 2 0 2 2 1 1 0 0 2 2 1 2 2 2 2 0 2 1 2 2 2 0 2 1
 2 2 1 2 2 1 1 1 1 0 1 2 1 2 2 2 2 2 0 2 0 1 0 1 1 2 2 2 0 2 2 1 1 1 1 2 2
 0 0 1 1 1 0 1 2 2 2 1 1 2 2 0 1 2 1 2 0 2 2 2 0 0 0 2 0 2 2 0 0 2 1 2 0 2
 0 1 2 1 1 2 2 0 1 2 1 1 2 2 2 1 2 1 2 2 2 2 1 1 1 2 1 2 0 0 1 0 2 1 2 0 1
 0 2 1 1 0 1 1 0 2 1 2 2 0 2 2 2 2 1 2 1 2 0 0 1 1 1 1 2 0 0 2 2 1 1 2 2 1
 1 2 0 0 0 1 1 2 0 2 2 1 1 2 1 1 1 2 1 0 0 1 2 2 1 2 2 0 1 1 1 2 2 1 1 1 1
 2 0 1 2 1 0 1 1 1 1 1 1 2 1 2 1 1 2 2 2 1 2 2 1 1 1 2 0 0 0 2 2 0 1 1 1 1
 2 1 1 0 0 0 1 1 0 0 2 2 2 1 0 1 1 1 0 1 1 2 2 2 0 2 1 0 1 1 1 0 1 2 2 0 0
 0 2 2 0 1 2 2 1 2 1 2 2 1 0 1 1 2 0 2 0 2 0 0 1 1 0 2 1 1 2 1 1 0 0 0 0 1
 0 1 1 2 1 1 0 1 0 0 0 2 1 1 0 1 1 1 2 2 1 1 1 1 2 1 1 1 2 2 2 1 0 2 1 0 2
 2 0 2 1 2 2 1 0 1 1 0 0 